In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import math
import time
import pandas as pd
import numpy as np

In [3]:
url_data = pd.read_csv("review_data.csv")

In [4]:
def collect_data(amazon_url, cr_url):
    comments = []
    date = []
    helpful = []
    ratings = []
    
    options = Options()
    options.add_argument('--kiosk')
    browser = webdriver.Chrome('chromedriver.exe', chrome_options=options)
    browser.get(amazon_url)
    
    num_reviews = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[4]/div/span").text
    num_reviews = int(num_reviews.split()[-3].replace(",", ""))
    num_pages = math.ceil(num_reviews/10)
    
    for i in range(1, num_pages):
        browser.get(amazon_url + "&pageNumber={}".format(i))
        time.sleep(2)
        for j in range(1, 12):
            try:
                text_1 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[4]/span/span".format(j)).text
                text_2 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/span".format(j)).text
                try:
                    text_3 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[7]/div/span[1]/div[1]/span".format(j)).text
                except:
                    text_3 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[5]/div/span[1]/div[1]/span".format(j)).text
                text_4 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[2]/a[1]/i/span".format(j)).get_attribute('innerHTML')
                comments.append(text_1)
                date.append(text_2)
                helpful.append(text_3)
                ratings.append(text_4)
                
            except:
                continue
            
    browser.get(amazon_url + "&pageNumber={}".format(num_pages))
    for num in range(1, num_reviews-(num_pages-1)*10 + 1):
        try:
            text_1 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[4]/span/span".format(num)).text
            text_2 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/span".format(num)).text
            try:
                text_3 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[7]/div/span[1]/div[1]/span".format(num)).text
            except:
                text_3 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[5]/div/span[1]/div[1]/span".format(num)).text
            text_4 = browser.find_element_by_xpath("/html/body/div[1]/div[3]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{}]/div/div/div[2]/a[1]/i/span".format(num)).get_attribute('innerHTML')
            comments.append(text_1)
            date.append(text_2)
            helpful.append(text_3)
            ratings.append(text_4)
        
        except:
            continue
    
    browser.get(cr_url)
    browser.find_element_by_xpath("/html/body/div[2]/div/header/div[1]/div[3]/div/div/div[1]/div/div/div/a/div[2]").click()
    browser.find_element_by_xpath("/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/form/div[1]/input").send_keys("94lyj@snu.ac.kr")
    time.sleep(2)
    browser.find_element_by_xpath("/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/form/div[2]/input").send_keys("94lyj@snu.ac.kr")
    browser.find_element_by_xpath("/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/form/div[6]/input").click()
    time.sleep(2)
    report_score = browser.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/div/div[2]/div[3]/div[1]/div[1]/div/div/div[1]/div[2]/span/div/p").text
    browser.quit()


    df = pd.DataFrame(list(zip(comments, date, ratings, helpful)),
                      columns=["comments", "date", "ratings", "helpful"])
    df['cs_score'] = report_score
    
    return df


def collect_price(cr_url):
    price = []
    options = Options()
    options.add_argument('--kiosk')
    browser = webdriver.Chrome('chromedriver.exe', chrome_options=options)
    browser.get(cr_url)
    browser.find_element_by_xpath("/html/body/div[2]/div/header/div[1]/div[3]/div/div/div[1]/div/div/div/a/div[2]").click()
    browser.find_element_by_xpath("/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/form/div[1]/input").send_keys("94lyj@snu.ac.kr")
    time.sleep(2)
    browser.find_element_by_xpath("/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/form/div[2]/input").send_keys("94lyj@snu.ac.kr")
    browser.find_element_by_xpath("/html/body/div[2]/div/div[2]/div/div/div/div/div[1]/form/div[6]/input").click()
    time.sleep(2)
    try:
        report_price = browser.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/div/div[2]/div[3]/div[2]/div[3]/div[1]/div/div/div[1]/span[2]").text
    except:
        report_price = browser.find_element_by_xpath("/html/body/div[4]/div/div/div[2]/div/div[2]/div[3]/div[2]/div[3]/div[1]/div/div/div[1]/div/span").text        
    browser.quit()
    price.append(report_price)
    
    df = pd.DataFrame(price, columns=['price'])
    return df

In [ ]:
for i in range(57):
    df = collect_data(url_data['amazon_link'][i], url_data['cr_link'][i])
    df['category'] = url_data['category'][i]
    df['product'] = url_data['Product'][i]
    
    df.to_csv('data{}.csv'.format(i+1), encoding="utf-8")
    
for i in range(58, len(url_data)):
    df = collect_data(url_data['amazon_link'][i], url_data['cr_link'][i])
    df['category'] = url_data['category'][i]
    df['product'] = url_data['Product'][i]
    
    df.to_csv('data{}.csv'.format(i+1), encoding="utf-8")

<ipython-input-15-b4250cbe4f55>:9: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('chromedriver.exe', chrome_options=options)


In [5]:
for i in range(57):
    df = collect_price(url_data['cr_link'][i])
    df['product'] = url_data['Product'][i]
    
    df.to_csv('price{}.csv'.format(i+1))

for i in range(58, len(url_data)):
    df = collect_price(url_data['cr_link'][i])
    df['product'] = url_data['Product'][i]
    
    df.to_csv('price{}.csv'.format(i+1))

<ipython-input-4-b4250cbe4f55>:76: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('chromedriver.exe', chrome_options=options)
